# Análise de condições climáticas em voos
![Avião durante tempestade](https://aeroin.net/wp-content/uploads/2020/07/Tempestade-Raio-20072301-1024x679.jpg)


### Importando bibliotecas

In [222]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime, timedelta
import random

### Gerando dataset de clima

In [223]:
date_range = pd.date_range(start=datetime(2006, 1, 1), end=datetime(2006, 12, 31), freq='D')

# Simulando dados meteorológicos
weather_data = {
    "Date": date_range,
    "Tempestade": [random.choice([0, 1]) for _ in range(len(date_range))],
    "Visibilidade": [random.uniform(0.5, 10.0) for _ in range(len(date_range))],
    "Vento": [random.uniform(20, 120) for _ in range(len(date_range))]
}

# Criando o DataFrame
weather_df = pd.DataFrame(weather_data)

# Salvando o DataFrame simulado para uso posterior
weather_df.to_csv('dados_meteorologicos_2006.csv', index=False)

# Visualizar as primeiras linhas do DataFrame meteorológico
weather_df.head()



,Date,Tempestade,Visibilidade,Vento
0,2006-01-01,0,2.414531,68.087416
1,2006-01-02,1,4.603092,25.966946
2,2006-01-03,1,1.166116,115.268901
3,2006-01-04,0,7.521685,37.646306
4,2006-01-05,1,8.701634,21.957825


### Juntando os datasets

In [224]:
df = pd.read_csv('../../../Datasets/2006-sample.csv')
df_weather = pd.read_csv('dados_meteorologicos_2006.csv')
df['Date'] =  df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-' + df['DayofMonth'].astype(str)
df['Date'] = pd.to_datetime(df['Date'])
df_full = pd.merge(df,weather_df, on=['Date', 'Date'])

## Análise da sazonalidade

### Análise sobre tempestades

In [225]:
df_storm = df_full[df_full['Tempestade']==1]
df_storm = df_storm.drop_duplicates(subset='DayofMonth')
storm_by_month = df_storm.groupby('Month').size()
print('Quantidade de dias em que houve tempestade por mês')
for month, number in storm_by_month.items():
    print(f'Mês {month}: {number} tempestades')
print(f'Com o mês {storm_by_month.idxmax()} sendo o mês com mais tempestades, tendo ao total {storm_by_month.max()}')

Quantidade de dias em que houve tempestade por mês
Mês 1: 5 tempestades
Mês 2: 4 tempestades
Mês 3: 2 tempestades
Mês 4: 3 tempestades
Mês 5: 1 tempestades
Mês 6: 1 tempestades
Mês 7: 4 tempestades
Mês 8: 2 tempestades
Mês 10: 2 tempestades
Mês 11: 3 tempestades
Mês 12: 4 tempestades
Com o mês 1 sendo o mês com mais tempestades, tendo ao total 5


### Análise sobre ventos fortes

In [226]:
df_windstorm = df_full[df_full['Vento']>89]
df_windstorm = df_windstorm.drop_duplicates(subset='DayofMonth')
windstorm_by_month = df_windstorm.groupby('Month').size()
print('Quantidade de ventanias fortes por mês')
for month, number in windstorm_by_month.items():
    print(f'Mês {month}: {number} ventanias')
print(f'Com o mês {windstorm_by_month.idxmax()} sendo o mês com mais ventanias, tendo ao total {windstorm_by_month.max()}')

Quantidade de ventanias fortes por mês
Mês 1: 4 ventanias
Mês 2: 2 ventanias
Mês 3: 4 ventanias
Mês 4: 3 ventanias
Mês 5: 1 ventanias
Mês 6: 1 ventanias
Mês 7: 4 ventanias
Mês 8: 4 ventanias
Mês 9: 2 ventanias
Mês 10: 1 ventanias
Mês 11: 1 ventanias
Mês 12: 4 ventanias
Com o mês 1 sendo o mês com mais ventanias, tendo ao total 4


### Análise sobre baixa visibilidade

In [227]:
df_visibility = df_full[df_full['Visibilidade']<5]
df_visibility = df_visibility.drop_duplicates(subset='DayofMonth')
visibility_by_month = df_visibility.groupby('Month').size()
print('Quantidade de dias com baixa visilibdade por mês')
for month, number in visibility_by_month.items():
    print(f'Mês {month}: {number} ventanias')
print(f'Com o mês {visibility_by_month.idxmax()} sendo o mês com mais ventanias, tendo ao total {visibility_by_month.max()}')

Quantidade de dias com baixa visilibdade por mês
Mês 1: 3 ventanias
Mês 2: 2 ventanias
Mês 3: 3 ventanias
Mês 4: 4 ventanias
Mês 5: 1 ventanias
Mês 6: 1 ventanias
Mês 7: 1 ventanias
Mês 8: 5 ventanias
Mês 9: 4 ventanias
Mês 10: 4 ventanias
Mês 11: 1 ventanias
Mês 12: 2 ventanias
Com o mês 8 sendo o mês com mais ventanias, tendo ao total 5


## Impacto das condições climáticas:

### Tempestade influencia atraso?

In [228]:
df_storm = df_full[df_full['Tempestade']==1]
delay_count = df_storm[df_storm['ArrDelay']>0].shape[0]
print(f"Quando ocorre uma tempestade {delay_count} voos são atrasados de um total de {len(df_storm['Tempestade'])}, representando {delay_count*100/len(df_storm['Tempestade']):.2f}%")
delay_mean = df_full[(df_full['Tempestade']==1) & (df_full['ArrDelay']>0)]
print(f"Com uma média de {delay_mean['ArrDelay'].mean():.2f} minutos de atraso")

Quando ocorre uma tempestade 17364 voos são atrasados de um total de 38650, representando 44.93%
Com uma média de 30.69 minutos de atraso


### Ventos acima de 89 km/h influenciam o atraso?

In [229]:
df_windstorm = df_full[df_full['Vento']>89]
delay_count = df_windstorm[df_windstorm['ArrDelay']>0].shape[0]
print(f"Quando ocorrem ventos acima de 89 km/h {delay_count} voos são atrasados de um total de {len(df_windstorm['Vento'])}, representando {delay_count*100/len(df_windstorm['Vento']):.2f}%")
delay_mean = df_full[(df_full['Vento']>0) & (df_full['ArrDelay']>0)]
print(f"Com uma média de {delay_mean['ArrDelay'].mean():.2f} minutos de atraso")

Quando ocorrem ventos acima de 89 km/h 9321 voos são atrasados de um total de 20746, representando 44.93%
Com uma média de 30.39 minutos de atraso


### Baixa visibilidade influencia o atraso?

In [230]:
df_visibility = df_full[df_full['Visibilidade']<5]
delay_count = df_visibility[df_visibility['ArrDelay']>0].shape[0]
print(f"Quando há baixa visibilidade {delay_count} voos são atrasados de um total de {len(df_visibility['Visibilidade'])}, representando {delay_count*100/len(df_visibility['Visibilidade']):.2f}%")
delay_mean = df_full[(df_full['Visibilidade']<5) & (df_full['ArrDelay']>0)]
print(f"Com uma média de {delay_mean['ArrDelay'].mean():.2f} minutos de atraso")

Quando há baixa visibilidade 15121 voos são atrasados de um total de 33251, representando 45.48%
Com uma média de 30.77 minutos de atraso


## Análise da eficência operacional

### Eficiência quando há tempestades

In [231]:
df_storm = df_full[df_full['Tempestade']==1]
delay_count = df_storm[df_storm['ArrDelay']<=0].shape[0]
print(f"Mesmo quando ocorre uma tempestade {delay_count} voos nâo são atrasados de um total de {len(df_storm['Tempestade'])}, representando {delay_count*100/len(df_storm['Tempestade']):.2f}%")

Mesmo quando ocorre uma tempestade 20597 voos nâo são atrasados de um total de 38650, representando 53.29%


### Eficiência quando há ventania

In [232]:
df_windstorm = df_full[df_full['Vento']>89]
delay_count = df_windstorm[df_windstorm['ArrDelay']<=0].shape[0]
print(f"Mesmo quando ocorrem ventos acima de 89 km/h {delay_count} voos não são atrasados de um total de {len(df_windstorm['Vento'])}, representando {delay_count*100/len(df_windstorm['Vento']):.2f}%")

Mesmo quando ocorrem ventos acima de 89 km/h 11051 voos não são atrasados de um total de 20746, representando 53.27%


### Eficiência quando há baixa visibilidade

In [233]:
df_visibility = df_full[df_full['Visibilidade']<5]
delay_count = df_visibility[df_visibility['ArrDelay']<=0].shape[0]
print(f"Mesmo quando há baixa visibilidade {delay_count} voos não são atrasados de um total de {len(df_visibility['Visibilidade'])}, representando {delay_count*100/len(df_visibility['Visibilidade']):.2f}%")

Mesmo quando há baixa visibilidade 17486 voos não são atrasados de um total de 33251, representando 52.59%
